In [42]:
def stitch_finder(text, targets):
    """
    Detect substrings using a sliding window.

    :param text: The input string to search.
    :param targets: A list of target substrings to detect.
    :return: A list of lists, each containing detected substrings with their numbers for each line.
    """
    # Normalize the text and targets for case-insensitive search
    text = text.lower()
    text_result = []
    targets = [target.lower() for target in targets]

    # Split the text into lines
    lines = text.splitlines()

    # Initialize results
    all_results = []

    # Sliding window approach
    window_size = max(len(target) for target in targets)  # Max length of target substrings

    smaller = False
    for line in lines:
        if line:
            line_results = []
            line += '   '
            for i in range(len(line) - window_size + 1):
                # Extract the current window
                window = line[i:i + window_size]
                #print(window)
                # Check for any target in the window
                for target in targets:
                    if window.startswith('[') or window.startswith('('):
                        toadd = []
                        smaller = True
                    elif window.startswith(']') or window.startswith(')'):
                        if toadd:
                            line_results.append(toadd)
                        toadd = []
                        smaller = False


                    if window.startswith(target):
                        # Ensure the characters before and after are not letters
                        before = line[i - 1] if i > 0 else " "
                        after = line[i + len(target)] if i + len(target) < len(line) else " "

                        if not before.isalpha() and not after.isalpha():
                        # Extract the number before the target
                            if target == "x":
                                num_start = i + 1
                                while num_start < len(line) and (line[num_start].isdigit() or line[num_start] == ' '):
                                    num_start += 1

                                number = line[i+1:num_start].strip()
                                result = f"{target}{number}".strip()
                                line_results.append(result)
                                break

                            else:
                                num_start = i - 1
                                while num_start >= 0 and (line[num_start].isdigit() or line[num_start] == ' '):
                                    num_start -= 1

                                number = line[num_start + 1:i].strip() if num_start < i - 1 else "1"

                                # Exclude matches where the characters before (excluding spaces) are "th", "nd", or "st"
                                context_start = max(0, num_start - 2)
                                context = line[context_start:num_start + 1].strip()

                                if context not in ["th", "nd", "st"]:
                                    if not number and target not in ["flo", "blo", "mr"]:
                                        number = "1"
                                    result = f"{number} {target}".strip()
                                    if smaller:
                                        toadd.append(result)
                                    else:
                                        line_results.append(result)
                                break  # Move to the next window after finding a match
            if line_results:
                all_results.append(line_results)

            line = line.replace('\t', ' ')[:-3]
            
            text_result.append(line)
        
    return (text_result, all_results)


targets = ["sc", "ch", "dc", "hdc", "tc", "mr", "inc", "dec", "blo", "flo", "slst", "sl st", "x"]


Matches found: (['r1         6 sc in a mr', 'r2         [inc] x 6  (12)', 'r3         [sc, inc] x 6 (18)', 'r4         16 sc, [sl st, ch1, dc, ch1, sl st] in flo of one stitch, 1 sc', 'if you need more help with round 4, please visit the video tutorial at timestamp: 10:52', 'r5         2 sc, [sl st, ch1, dc ch1, sl st] in  flo of one stitch, 15 sc (the 14th sc should be made in the blo of r3)', 'r6         7 sc (the third sc should be made in the blo of r4), 4-dcbo , 3 sc,  4-dcbo, 6 sc (18)', 'r7-9 18 sc (18)   3 rounds', 'r10         8 sc, 4-dcbo, 3 sc, 4-dcbo, 5 sc (18)'], [['6 sc', 'mr'], [['1 inc'], 'x6'], [['1 sc', '1 inc'], 'x6'], [['1 sl st', '1 ch', '1 dc', '1 ch', '1 sl st'], 'flo', '1 sc'], ['2 sc', ['1 sl st', '1 ch', '1 dc', '1 ch', '1 sl st'], 'flo', '15 sc'], [['1 sc', 'blo'], '3 sc', '6 sc'], ['8 sc', '3 sc', '5 sc']])
